In [ ]:
# 1、推导式与函数式
ages = [-1, 10, 20]
sum([age if age >=0 else 0 for age in ages ])
sum(age if age >=0 else 0 for age in ages)  # 生成器
# 函数式: map, filter, reduce等
sum(map(lambda age: age if age >=0 else 0, ages))

算法

In [16]:
# 排序
peoples = [
    {"age": 1, "name": "kehan"},
    {"age": 24, "name": "lwy"},
    {"age": 25, "name": "skycrab"},
]
# age从小到大,
sorted(peoples, key=lambda p:p["age"])
# 从大到小
import operator
sorted(peoples, key=operator.itemgetter("age"),reverse=True)
'''
operator.attrgetter     # 属性
operator.itemgetter     #元素
operator.methodcaller   #方法
'''

d1={1:'a',3:'c',4:'d'}
# 按字典的键排序
print(sorted(d1,reverse=True))
sorted(d1.keys())
# 按字典值排序
print(sorted(d1.items(), key=lambda x:x[1]))

[4, 3, 1]
[(1, 'a'), (3, 'c'), (4, 'd')]


反射(自省)
getattr, setattr, hasattr, dir

__dict__, __slots__

callable, isinstance

traceback

inspect

好处是什么？？？

orm

In [22]:
# 装饰器
def XXX(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        # before hook
        return func(*args, **kwargs)
        # after hook
    return wrapper


def super_required(view_func):
    """超级用户控制"""
    @wraps(view_func)
    def decorator(request, *args, **kwargs):
        if request.user.is_authenticated() and request.user.is_superuser:
            return view_func(request,*args,**kwargs)
        else:
            return HttpResponseRedirect("/login/?next= {0}".format(request.path))
    return decorator
                                        
# 属性拦截与装饰器结合
def log(level):
    """记录日志"""
    assert level in ("debug", "info", "warn", "error")
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            start = time.time()
            result = func(*args, **kwargs)
            write = getattr(logger, level)
            write("sql: %s\nrun time: %s seconds",
                args, time.time()-start)
            return result
        return wrapper
    return decorator

In [23]:
# 上下文管理器 with
class cd(object):
    def __init__(self, path):
        self.src = os.getcwd()
        self.dest = path

    def __enter__(self):
        os.chdir(self.dest)

    def __exit__(self, exc_type, exc_val, exc_tb):
        """返回True忽略异常"""
        os.chdir(self.src)

In [1]:
# gevent  借助gevent让flask(django)拥有实时推送能力(对比tornado)
from gevent.server import StreamServer

def handle(socket, address):
    socket.send("Hello from a telnet!\n")
    for i in range(5):
        socket.send(str(i) + '\n')
    socket.close()

server = StreamServer(('127.0.0.1', 5000), handle)
server.serve_forever()

# 定时任务
def f():
    print time.time()
    print 'eeeee'
from gevent.core import loop
l = loop()
timer = l.timer(2,3) #2秒后启动，3秒后再次启动
print time.time()
timer.start(f)
l.run()

SyntaxError: invalid syntax (<ipython-input-1-6107aae69357>, line 15)

In [7]:
def swap(data, root, last):
    data[root], data[last] = data[last], data[root]

#调整父节点 与孩子大小， 制作大顶堆
def adjust_heap(data, par_node, high):

    new_par_node = par_node
    j = 2*par_node +1   #取根节点的左孩子， 如果只有一个孩子 high就是左孩子，如果有两个孩子 high 就是右孩子

    while j <= high: #如果 j = high 说明没有右孩子，high就是左孩子
        if j < high and data[j] < data[j+1]: #如果这儿不判断 j < high 可能超出索引
            # 一个根节点下，如果有两个孩子，将 j  指向值大的那个孩子
            j += 1
        if data[j] > data[new_par_node]: #如果子节点值大于父节点，就互相交换
            data[new_par_node], data[j] = data[j], data[new_par_node]
            new_par_node = j #将当前节点，作为父节点，查找他的子树
            j = j * 2 + 1

        else:
            # 因为调整是从上到下，所以下面的所有子树肯定是排序好了的，
            #如果调整的父节点依然比下面最大的子节点大，就直接打断循环，堆已经调整好了的
            break


# 索引计算: 0 -->1 --->....
#    父节点 i   左子节点：2i +1  右子节点：2i +2  注意：当用长度表示最后一个叶子节点时 记得 -1
#    即 2i + 1 = length - 1 或者 2i + 2 = length - 1
#    2i+1 + 1 = length 或 2i+2 + 1 = length
#    2(i+1)=length 或 2(i+1）+1 = length
#    设j = i+1  则左子节点(偶数)：2j = length 和 右子节点(基数)：2j+1 = length
#    2j//2 = j == (2j+1)//2 这两个的整除是一样的，所以使用length//2 = j 然后 i + 1 = j
#    i = j-1  = length//2 -1  #注意左子节点:2i+1 //2 =i  而右子节点：(2i+2)//2 = i+1 

# 从第一个非叶子节点(即最后一个父节点)开始，即 list_.length//2 -1（len(list_)//2 - 1）

# 开始循环到 root 索引为：0 的第一个根节点， 将所有的根-叶子 调整好，成为一个 大顶堆
def heap_sort(lst):
    """
    根据列表长度，找到最后一个非叶子节点，开始循化到 root 根节点，制作 大顶堆
    :param lst: 将列表传入
    :return:
    """
    length = len(lst)
    last = length -1  #最后一个元素的 索引
    last_par_node = length//2 -1

    while last_par_node >= 0:

        adjust_heap(lst, last_par_node, length-1)
        last_par_node -= 1  #每调整好一个节点，从后往前移动一个节点

    # return lst

    while last > 0:
        #swap(lst, 0, last)
        lst[0], lst[last] = lst[last],lst[0]
        # 调整堆让 adjust 处理，最后已经排好序的数，就不处理了
        adjust_heap(lst, 0, last-1)
        last -= 1

    return lst #将列表返回

import time,random
def sift_down(arr, node, end):
    root = node
    #print(root,2*root+1,end)
    while True:
        # 从root开始对最大堆调整
        child = 2 * root +1  #left child
        if child  > end:
            #print('break',)
            break
        #print("v:",root,arr[root],child,arr[child])
        #print(arr)
        # 找出两个child中交大的一个
        if child + 1 <= end and arr[child] < arr[child + 1]: #如果左边小于右边
            child += 1 #设置右边为大
        if arr[root] < arr[child]:
            # 最大堆小于较大的child, 交换顺序
            tmp = arr[root]
            arr[root] = arr[child]
            arr[child]= tmp
            # 正在调整的节点设置为root
            #print("less1:", arr[root],arr[child],root,child)
            root = child #
            #[3, 4, 7, 8, 9, 11, 13, 15, 16, 21, 22, 29]
            #print("less2:", arr[root],arr[child],root,child)
        else:
            # 无需调整的时候, 退出
            break
    #print(arr)
    #print('-------------')

def heap_sort2(arr):
    # 从最后一个有子节点的孩子还是调整最大堆
    first = len(arr) // 2 -1
    for i in range(first, -1, -1):
        sift_down(arr, i, len(arr) - 1)
    #[29, 22, 16, 9, 15, 21, 3, 13, 8, 7, 4, 11]
    # print('--------end---',arr)
    # 将最大的放到堆的最后一个, 堆-1, 继续调整排序
    for end in range(len(arr) -1, 0, -1):
        arr[0], arr[end] = arr[end], arr[0]
        sift_down(arr, 0, end - 1)
        #print(arr)

if __name__ == '__main__':
    list_ = list(range(10000,1,-1))
    random.shuffle(list_)
    heap_sort2(list_)
    #print(list_)


#最后结果为：
# [3, 4, 7, 8, 9, 11, 13, 15, 16, 21, 22, 29]

In [2]:
# 上下文管理，threading.local对象，用于为每个线程开辟一块空间来保有他独特的值
import threading
 
local_values = threading.local()
 
 
def func(num):
    local_values.name = num
    import time
    time.sleep(1)
    print(local_values.name, threading.current_thread().name)
 
 
for i in range(20):
    th = threading.Thread(target=func, args=(i,), name='线程%s' % i)
    th.start()

1 线程1
6 线程6
4 线程4
0 82 线程2
7 线程7
13 10 线程10
 线程8
线程05 线程5
线程1314 线程14
18 线程18

12 线程12
3
9 线程9
 线程3
17 线程17
19 线程19
16 线程16
11 线程11
15 线程15


In [4]:
# 上下文管理，threading.local 协程
import threading
try:
    from greenlet import getcurrent as get_ident # 协程
except ImportError:
    try:
        from thread import get_ident
    except ImportError:
        from _thread import get_ident
class UsualLocal(object):
    def __init__(self):
        self.storage={}
        self.get_ident=get_ident
    
    def set(self,k,v):
        ident=self.get_ident()
        origin=self.storage.get(ident)
        if not origin:
            origin={k:v}
        else:
            origin[k]=v
        self.storage[ident]=origin
    
    def get(self,k):
        ident=self.get_ident()
        origin=self.storage.get(ident)
        if not origin:
            return None
        return origin.get(k,None)

class NewLocal(object):
    def __init__(self):
        #self.storage={}
        object.__setattr__(self,'storage',{})     # 对象.XX 会触发 __setattr__方法，object.__setattr__不会循环触发
        #self.get_ident=get_ident
        object.__setattr__(self,'get_ident',get_ident)
    
    def __setattr__(self,k,v):
        ident=self.get_ident()
        origin=self.storage.get(ident)
        if not origin:
            origin={k:v}
        else:
            origin[k]=v
        self.storage[ident]=origin
    
    def __getattr__(self,k):
        try:
            return self.storage[self.get_ident()][k]
        except KeyError:
            raise AttributerError(k)

In [7]:
# 偏函数 functools.partial
import functools
def func(a1,a2,a3):
    print(a1,a2,a3)

new_func=functools.partial(func,1,2)
new_func(3)

1 2 3


In [ ]:
# 数据库连接池
import pymysql
from DBUtils.PooledDB import PooledDB, SharedDBConnection
POOL = PooledDB(
    creator=pymysql,  # 使用链接数据库的模块
    maxconnections=6,  # 连接池允许的最大连接数，0和None表示不限制连接数
    mincached=2,  # 初始化时，链接池中至少创建的空闲的链接，0表示不创建
    maxcached=5,  # 链接池中最多闲置的链接，0和None不限制
    maxshared=3,  # 链接池中最多共享的链接数量，0和None表示全部共享。PS: 无用，因为pymysql和MySQLdb等模块的 threadsafety都为1，所有值无论设置为多少，_maxcached永远为0，所以永远是所有链接都共享。
    blocking=True,  # 连接池中如果没有可用连接后，是否阻塞等待。True，等待；False，不等待然后报错
    maxusage=None,  # 一个链接最多被重复使用的次数，None表示无限制
    setsession=[],  # 开始会话前执行的命令列表。如：["set datestyle to ...", "set time zone ..."]
    ping=0,
    # ping MySQL服务端，检查是否服务可用。# 如：0 = None = never, 1 = default = whenever it is requested, 2 = when a cursor is created, 4 = when a query is executed, 7 = always
    host='127.0.0.1',
    port=3306,
    user='root',
    password='123',
    database='pooldb',
    charset='utf8'
)

class SQLHelper(object):
    
    @staticmethod          #静态方法，使用时不需要实例化对象
    def fetch_one(self,sql,args):
        conn=Pool.connection()
        cursor=conn.cursor()
        cursor.execute(sql,args)
        result=cursor.fetchone()
        conn.close()
        return result
    
    @staticmethod          #静态方法，使用时不需要实例化对象
    def fetch_all(self,sql,args):
        conn=Pool.connection()
        cursor=conn.cursor()
        cursor.execute(sql,args)
        result=cursor.fetchall()
        conn.close()
        return result

In [5]:
class Mymeta(type):
    def __init__(self,name,bases,dic):
        super().__init__(name,bases,dic)
        # 2
        print('----Mymeta.__init__',self.__name__,dic,self.yaml_tag)
    
    def __new__(cls,*args,**kwargs):
        # 1
        print('Mymeta.__new__',cls.__name__)
        return type.__new__(cls,*args,**kwargs)
    
    def __call__(cls,*args,**kwargs):
        print('---Mymeta.__call__')
        obj=cls.__new__(cls)
        cls.__init__(cls,*args,**kwargs)
        return obj
    
class Foo(metaclass=Mymeta):
    yaml_tag = '!Foo'
    def __init__(self, name):
        print('Foo.__init__')
        self.name = name
    
    def __new__(cls, *args, **kwargs):
        print('Foo.__new__')
        return object.__new__(cls)

Mymeta.__new__ Mymeta
----Mymeta.__init__ Foo {'__module__': '__main__', '__qualname__': 'Foo', 'yaml_tag': '!Foo', '__init__': <function Foo.__init__ at 0x000002C8AAFF12F0>, '__new__': <function Foo.__new__ at 0x000002C8AAFF1378>} !Foo


In [6]:
foo = Foo('foo')

---Mymeta.__call__
Foo.__new__
Foo.__init__


从上面的运行结果可以发现在定义 class Foo() 定义时，会依次调用 MyMeta 的 __new__ 和 __init__ 方法构建 Foo 类，然后在调用 foo = Foo() 创建类的实例对象时，才会调用 MyMeta 的 __call__ 方法来调用 Foo 类的 __new__ 和 __init__ 方法。

把上面的例子运行完之后就会明白很多了，正常情况下我们在父类中是不能对子类的属性进行操作，但是元类可以。换种方式理解：元类、装饰器、类装饰器都可以归为元编程。

In [1]:
bin(ord('h'))

'0b1101000'